## Пример на основе RoCoLa
### В качестве теста выступает валидация, в качестве примеров для few-shot выступает трейн.

In [1]:
!cd ../

In [2]:
from llmtf.base import Task, SimpleFewShotHFTask, LLM
from sklearn.metrics import matthews_corrcoef
from tqdm import tqdm
from typing import Dict, List, Tuple
from datasets import load_dataset, Dataset
import copy
from llmtf.metrics import mean

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Для создания новой таски необходимо, чтобы она была наследником класса Task с реализованными функциями

```python
def __init__(self, **kwargs):
    # Важно определить как минимум 2 переменные: self.method и self._max_new_token.
    # self.method - Это режим работы. Либо calculate_tokens_proba либо generate. Метод, который будет вызываться у LLM. 
    # self._max_new_token - дефолтное значение 64, если нужно генерить больше или меньше, стоит поменять на нужное значение. Для calculate_tokens_proba достаточно 1 токена.

    self.method = 'calculate_tokens_proba'
    self._max_new_tokens = 1

@property
def name(self) -> str:
    # Возвращает имя датасета, которое будет записываться потом в логи.

@property
def choices(self) -> List:
    # ТОЛЬКО для случая calculate_tokens_proba. Вероятность на каких токенах оцениваем?

def evaluate(self, sample, y_pred) -> Dict:
    # Принимает на вход результат пару sample и LLM.method. По ним необхоимо посчитать критерий правильности результата работы модели и вернуть его (или несколько) как Dict
    # Например {'acc': sample['outputs'] == y_pred}. Имя 'acc' будет потом использоваться в методе aggregation.

def aggregation(self) -> Dict:
    # Возвращает Dict из имя -> Callable. В дальнейшем этот Callable вызывается для списка результатов, в котором каждый обработан методом evaluate.
    # Например {"acc": np.mean} означает, что в метод np.mean будет подан список из результата evaluate по ключу "acc".

def leaderboard_aggregation(self, metrics: Dict) -> float:
    # итоговая метрика по датасету. Это может быть mean, или определенная выбранная метрика. На вход идет Dict со значениями метрик, например, {"acc": 0.5, "f1": 0.5}, необходимо вернуть число.

def load_dataset(self, **kwargs) -> Tuple[List[Dict], List[Dict]]:
    # Основная логика подготовки данных тут. load_dataset возвращает пару списков: messages и samples. То есть задача объекта класса Task в том, чтобы преобразовать вашу задачу в формат сообщений.
```

### Для упрощения добавления новой таски реализован класс SimpleFewShotHFTask, который последовательно проходит по huggingface dataset и формирует список messages. При наследовании от него необходимо реализовать несколько простых функций:

```python
def dataset_args(self) -> Dict:
    # передается в метод datasets.load_dataset(**self.dataset_args())

def test_split_name(self) -> str:
    # какой сплит мы считаем за тестовый?

def prompt_split_name(self) -> str:
    # из какого сплита мы формируем few-shot примеры?

def create_messages(self, sample, with_answer) -> List[Dict]:
    # по сути главный метод. Преобразует пример из датасета (строчку) в набор сообщений. with_answer - для случая, когда мы можем добавть ответ в сообщения, например, когда формируем few-shot примеры.
```
#### и опционально:
```python
def prompt_dataset_start_idx(self) -> int:
    # первый индекс промпт сплита для формирования few-shot примеров. Они используются один за одним последовательно начиная с возвращаемого тут параметра. В дефолтной реализации = 0.
```

### Теперь посмотрим на RoCoLa
Каждый sample датасета в [RuCoLa](https://huggingface.co/datasets/RussianNLP/rucola) состоит из нескольких полей, из которых нам интересны sentence и label. На их основе и будет формироваться messages.

In [3]:
class RuColaCustomTask(SimpleFewShotHFTask):
    RUCOLA_HF_PATH = 'RussianNLP/rucola'
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.method = 'calculate_tokens_proba'
        self._max_new_tokens = 1

    def evaluate(self, sample, y_pred) -> Dict:
        y_true = str(sample['label'])
        y_pred = sorted([pair for pair in y_pred.items()], key=lambda x: -x[1])[0][0]
        return {"acc": y_true == y_pred, "mcc": [y_true, y_pred]}

    @property
    def name(self):
        return 'russiannlp/rucola_custom'
    
    @property
    def choices(self):
        return ["0", "1"]

    def aggregation(self) -> Dict:
        return {"acc": mean, "mcc": lambda data: matthews_corrcoef([d[0] for d in data], [d[1] for d in data])}

    def dataset_args(self) -> Dict:
        return {'path': self.RUCOLA_HF_PATH}

    def test_split_name(self) -> str:
        return 'validation'

    def prompt_split_name(self) -> str:
        return 'train'

    def create_messages(self, sample, with_answer):
        messages = []
        instruction_user = 'Твоя задача определить приемлемость текста для русского языка с точки зрения синтаксиса, морфологии и семантики. Ответом должно служить одно число: 0 или 1, где 0 - предложение не приемлемо с точки зрения русского языка, 1 - приемлемо.\nТекст: {sentence}'
        instruction_bot = 'Ответ: {label}'
        instruction_bot_incomplete = 'Ответ:'

        bot_content = instruction_bot.format(**sample) if with_answer else instruction_bot_incomplete

        messages.append({'role': 'user', 'content': instruction_user.format(**sample)})
        messages.append({'role': 'bot', 'content': bot_content})

        return messages

    def prompt_dataset_start_idx(self) -> int:
        # в ближайших индексах после 29 сбалансировано по меткам классов, вот поэтому
        return 29

### Этот код можно (как уже и сделано) добавить в поддиректорию tasks и затем записать в объект TASK_REGISTRY в ```tasks.__init__```. Но помимо этого можно воспользоваться методом add_new_task в Evaluator во время исполнения, как будет показано далее.

In [4]:
from llmtf.evaluator import Evaluator

evaluator = Evaluator()
evaluator.add_new_task('OurRuColaCustomTask', RuColaCustomTask)

### Инициализация LLM. В данном случае с использованием VLLM фреймворка.

In [5]:
from llmtf.model import VLLMModel

conv_path = 'conversation_configs/openchat_3.5_1210.json'
model_name_or_path = 'openchat/openchat-3.5-0106'
model = VLLMModel(conv_path, device_map='cuda:0', disable_sliding_window=True, enable_prefix_caching=True)
model.from_pretrained(model_name_or_path)

WARNING 06-10 17:34:23 ray_utils.py:46] Failed to import Ray with ModuleNotFoundError("No module named 'ray'"). For multi-node inference, please install Ray with `pip install ray`.
INFO 06-10 17:34:23 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='openchat/openchat-3.5-0106', speculative_config=None, tokenizer='openchat/openchat-3.5-0106', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda:0, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=openchat/openchat-3.5-0106)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should se

INFO 06-10 17:34:24 selector.py:51] Using XFormers backend.
INFO 06-10 17:34:26 selector.py:51] Using XFormers backend.
INFO 06-10 17:34:26 weight_utils.py:207] Using model weights format ['*.safetensors']
INFO 06-10 17:34:29 model_runner.py:146] Loading model weights took 13.4917 GB
INFO 06-10 17:34:29 gpu_executor.py:83] # GPU blocks: 29181, # CPU blocks: 2048
INFO 06-10 17:34:31 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-10 17:34:31 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-10 17:34:39 model_runner.py:924] Graph capturing finished in 8 secs.
INFO 06-10 17:34:39 block_manager_v1.py:2

INFO: 2024-06-10 17:34:40,334: llmtf.base.llm: Override eos_token_id in generation_config from 32000 to 32000
INFO: 2024-06-10 17:34:40,335: llmtf.base.llm: Model id: openchat/openchat-3.5-0106
INFO: 2024-06-10 17:34:40,336: llmtf.base.llm: _check_if_leading_space: "[28705, 28740]"
INFO: 2024-06-10 17:34:40,356: llmtf.base.llm: global_prefix = <s>
INFO: 2024-06-10 17:34:40,356: llmtf.base.llm: vllm_adds_bos = True
INFO: 2024-06-10 17:34:40,357: llmtf.base.llm: Resetting generation_config.stop_strings to []
INFO: 2024-06-10 17:34:40,357: llmtf.base.llm: Leading space: True


In [6]:
output_dir = 'examples/example_rucula_custom_openchat_3.5_0106_eval'
datasets_names = ['OurRuColaCustomTask']
evaluator.evaluate(model, output_dir, datasets_names=datasets_names, max_len=4000, few_shot_count=5, batch_size=8, max_sample_per_dataset=200)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 294.67it/s]
INFO: 2024-06-10 17:34:44,207: llmtf.base.russiannlp/rucola_custom: Loading Dataset: 3.84s
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:06<00:00,  3.92it/s]
INFO: 2024-06-10 17:34:50,594: llmtf.base.russiannlp/rucola_custom: Processing Dataset: 6.39s
INFO: 2024-06-10 17:34:50,594: llmtf.base.russiannlp/rucola_custom: Results for russiannlp/rucola_custom:
INFO: 2024-06-10 17:34:50,598: llmtf.base.llm: Resetting generation_config.stop_strings to []
INFO: 2024-06-10 17:34:50,598: llmtf.base.russiannlp/rucola_custom: {'acc': 0.735, 'mcc': 0.21467076918233885}
